<a href="https://colab.research.google.com/github/nebyu08/nlp_zth/blob/main/simple_bigram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [5]:
text=open("/content/input.txt").read()
chars=sorted(list(set(text)))
vocab_size=len(chars)

In [6]:
stoi={s:i for i,s in enumerate(chars)}
itos={i:s for s,i in stoi.items()}
encode=lambda s:[stoi[i] for i in s] # ecodes the string into integers
decode=lambda n:"".join([itos[i] for i in n]) #decode the number into strings

In [7]:
print(encode("hello there friend"))

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43, 1, 44, 56, 47, 43, 52, 42]


In [8]:
print(decode([46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43, 1, 44, 56, 47, 43, 52, 42]))

hello there friend


In [9]:
data=torch.tensor(encode(text),dtype=torch.long)
print(data.type,data.shape)

<built-in method type of Tensor object at 0x7b97cc3630b0> torch.Size([1115394])


In [10]:
# lets split our data into training and validating set
n=int(len(data)*0.9)
train_data=data[:n]
valid_data=data[n:]

In [11]:
block_size=8
x=train_data[:block_size]
y=train_data[1:block_size+1]

for i in range(block_size):
  context=x[:i+1]
  target=y[i]
  print("context |",context," target|",target)

context | tensor([18])  target| tensor(47)
context | tensor([18, 47])  target| tensor(56)
context | tensor([18, 47, 56])  target| tensor(57)
context | tensor([18, 47, 56, 57])  target| tensor(58)
context | tensor([18, 47, 56, 57, 58])  target| tensor(1)
context | tensor([18, 47, 56, 57, 58,  1])  target| tensor(15)
context | tensor([18, 47, 56, 57, 58,  1, 15])  target| tensor(47)
context | tensor([18, 47, 56, 57, 58,  1, 15, 47])  target| tensor(58)


In [12]:
# lets batchify our code
torch.manual_seed(1337)
batch_size=4
block_size=8
def get_batch(split):
  #this creates x batch and y batch
  data= train_data if split=="train" else valid_data
  xi=torch.randint(len(data)-block_size,(batch_size,))
  xb=torch.stack([data[i:i+block_size] for i in xi])
  yb=torch.stack([data[i+1:i+block_size+1] for i in xi])
  return xb,yb

In [13]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [58]:
# lets make a bigam
import torch.nn as nn
from torch.nn import functional as f
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size):   # super cool inheritance is going on here
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)

  def forward(self,idx,target=None):
    logits=self.token_embedding_table(idx) #the shape is batch size,context size,vocab size
    if target is None:
      loss=None
    else:
      B,T,C=logits.shape
      logits=logits.view(B*T,C)
      target=target.view(B*T)  #i could say -1 to make it flatten
      loss=F.cross_entropy(logits,target)
    return logits,loss

  def generate(self,idx,max_tokens):
    for _ in range(max_tokens):
      logits,loss=self(idx) # need only the logits
      logits=logits[:,-1,:] #make the probability distribution only
      probs=torch.softmax(logits,dim=1)  #i have to make this one probability distribution
      idx_next=torch.multinomial(probs,num_samples=1)
      idx=torch.cat((idx,idx_next),dim=1)
    return idx


In [59]:
m=BigramLanguageModel(vocab_size)
logits,loss=m(xb,yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx=torch.zeros(1,1,dtype=torch.long),max_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [60]:
# lets try to optimize the code
optimizer=torch.optim.AdamW(m.parameters(),lr=0.01)

In [76]:
for _ in range(1500):
  xb,yb=get_batch("train")
  logits,loss=m(xb,yb)
  #backward pass
  loss.backward()
  optimizer.zero_grad(set_to_none=True)
  optimizer.step()
print(loss.item())

2.832476854324341


In [78]:
print(decode(m.generate(idx=torch.zeros(1,1,dtype=torch.long),max_tokens=400)[0].tolist()))


HESCESCooourd har:

Thed, d'd.

Whar t o ore pus t ll s, oore tewe t
SCinou's, d ay bor tsth vespul f ard.
Whar hadu,
O:

Vin arsu, bucoute thinthays t ad.
Lethelt ale t ar ous tens! ore pr bu,
VILou,

Nor seanot t:
Whayoo altheeming, halallo ryoullal l O,
LIELIf t pris, ar d t oul an t an avinimauladextuk ve lmy, ounthefe ad.
Thalay y thay ou awank digh t
O, oor d Be t ar O, tsoutn Yoor myoreway.
